In [ ]:
'''Проект, прогнозирующий популярность 
продуктов на основе данных из Google Trends'''
from pytrends.request import TrendReq
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import time

def get_trends_data(pytrends, product, timeframe='today 5-y'):
    try:
        time.sleep(5)
        pytrends.build_payload([product], cat=0, timeframe=timeframe, geo='', gprop='')
        data = pytrends.interest_over_time()
        
        if data.empty:
            print(f"Нет данных для {product}")
            return None
        
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])
        
        return data
    except Exception as e:
        print(f"Ошибка при загрузке данных для {product}: {e}")
        return None

# Подключение к Google Trends
pytrends = TrendReq(hl='en-US', tz=360, retries=3, backoff_factor=0.3)

# Список продуктов для анализа
products = ["smartphone", "sneakers", "laptop"]

# Загрузка данных о трендах
all_data = {}
for product in products:
    print(f"Загружаем данные для: {product}")
    product_data = get_trends_data(pytrends, product)
    if product_data is not None:
        all_data[product] = product_data

# Построение прогнозов для всех продуктов
plt.figure(figsize=(15, 10))

for product, data in all_data.items():
    print(f"Строим прогноз для: {product}")
    
    # Подготовка данных
    df = data.reset_index()
    df.columns = ['ds', 'y']  # Для совместимости с Prophet
    
    try:
        # Создание и обучение модели
        model = Prophet()
        model.fit(df)
        
        # Прогнозирование
        future = model.make_future_dataframe(periods=365)
        forecast = model.predict(future)
        
        # Визуализация прогноза
        plt.subplot(len(all_data), 1, list(all_data.keys()).index(product) + 1)
        model.plot(forecast)
        plt.title(f"Прогноз популярности: {product.capitalize()}")
        plt.xlabel("Дата")
        plt.ylabel("Популярность")
        plt.grid()
        
        # Сохранение прогноза
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].to_csv(f"{product}_forecast.csv", index=False)
        print(f"Прогноз для {product} сохранён в файл: {product}_forecast.csv")
    
    except Exception as e:
        print(f"Ошибка при прогнозировании для {product}: {e}")

plt.tight_layout()
plt.show()

# Сравнительный график популярности всех продуктов
plt.figure(figsize=(12, 8))
for product, data in all_data.items():
    plt.plot(data.index, data[product], label=product.capitalize())
plt.title("Сравнение популярности продуктов")
plt.xlabel("Дата")
plt.ylabel("Популярность")
plt.legend()
plt.grid()
plt.show()